# ECS713: Week 11 

# Monads - The Log Monad

## Learning Objectives

By the end of this notebook you should: 
- understand the construction of the Log monad
- have seen examples of its use

## The basic construction

The Log monad allows the logging of computations. This is a simple example. 

As is normal with monads, we begin by defining the basic type constructor as a datatype. In this instance we have a simple form of the Log monad in which the Log is just structured as a String. There are other possibilities. It could be structured, for example, as a list of entries, where the entries themselves could either be Strings or possibly instances of a more specialised datatype. 

In [3]:
data Log a = Log String a deriving (Eq,Show)

The next two functions are defined for any monad. They are in the standard base (in this case in Control.Monad), but are not in the Prelude. They implement sequential application and the maps function. See http://hackage.haskell.org/package/base-4.14.0.0/docs/Control-Monad.html for documentation: 

In [4]:
ap mf mx = do {x <- mx; f <- mf; return (f x)}
liftM f mx = do {x <- mx; return (f x)}

Line 2: Use <$>
Found:
do x <- mx
   return (f x)
Why not:
do f <$> mx

Now we need to turn the Log datatype into an instance of Monad, and of course also Applicative and Functor with standard boilerplate to do that. 

In [5]:
-- make Monad out of Log
instance Monad Log where
-- compose Logs by concatenating entries, with separator "; ""
  (Log currentLog a) >>= f =
    let Log newEntry b = f a in Log (currentLog++newEntry) b
-- return has empty string for Log 
  return a = Log "" a

-- standard boilerplate to get Applicative 
instance Applicative Log where
  pure = return 
  (<*>) = ap

-- standard boilerplate to get Functor 
instance Functor Log where
  fmap = liftM

Line 7: Eta reduce
Found:
return a = Log "" a
Why not:
return = Log ""

## First use of the Log Monad

We can use the log monad to log a sequence of actions:

In [6]:
do
  x0 <- Log ("initial value = "++(show 0)++"; ") 0
  x1 <- Log ("add "++(show 3)++"; ") (x0+3)
  Log ("multiply by "++(show 2)++"; ") (x1*2)

Line 2: Redundant bracket
Found:
(show 0) ++ "; "
Why not:
show 0 ++ "; "Line 3: Redundant bracket
Found:
(show 3) ++ "; "
Why not:
show 3 ++ "; "Line 4: Redundant bracket
Found:
(show 2) ++ "; "
Why not:
show 2 ++ "; "

Log "initial value = 0; add 3; multiply by 2; " 6

The actions here are `start with 0`, `add 3` and `multiply by 2`, and can be seen on the right. Note there is no `return` statement, but we could give add that (redundantly): 

In [7]:
do
  x0 <- Log ("initial value = "++(show 0)++"; ") 0
  x1 <- Log ("add "++(show 3)++"; ") (x0+3)
  x2 <- Log ("multiply by "++(show 2)++"; ") (x1*2)
  return x2

Line 1: Redundant return
Found:
do x0 <- Log ("initial value = " ++ (show 0) ++ "; ") 0
   x1 <- Log ("add " ++ (show 3) ++ "; ") (x0 + 3)
   x2 <- Log ("multiply by " ++ (show 2) ++ "; ") (x1 * 2)
   return x2
Why not:
do x0 <- Log ("initial value = " ++ (show 0) ++ "; ") 0
   x1 <- Log ("add " ++ (show 3) ++ "; ") (x0 + 3)
   Log ("multiply by " ++ (show 2) ++ "; ") (x1 * 2)Line 2: Redundant bracket
Found:
(show 0) ++ "; "
Why not:
show 0 ++ "; "Line 3: Redundant bracket
Found:
(show 3) ++ "; "
Why not:
show 3 ++ "; "Line 4: Redundant bracket
Found:
(show 2) ++ "; "
Why not:
show 2 ++ "; "

Log "initial value = 0; add 3; multiply by 2; " 6

To make things clearer, let's wrap these actions up in some functions: 

In [10]:
-- three functions to log inititalisation, addition and multiplication
logInit :: Show a => a -> Log a
logInit n = Log ("initial value = "++ show n ++"; ") n

logAdd :: (Show a, Num a) => a -> a -> Log a 
logAdd m n = Log ("add "++ show m ++"; ") (n+m)

logMult :: (Show a, Num a) => a -> a -> Log a 
logMult m n = Log ("multiply by "++ show m ++ "; ") (n*m)

So our code becomes: 

In [12]:
do
  x <- logInit 0
  y <- logAdd 3 x
  logMult 2 y

Log "initial value = 0; add 3; multiply by 2; " 6

And if we put it into the bind pipeline style: 

In [13]:
logInit 0 >>= logAdd 3 >>= logMult 2

Log "initial value = 0; add 3; multiply by 2; " 6

Write code to log the following sequence of actions: 
- start with 3
- add 6 
- multiply by 8
- subtract 2

## Another use: `foldr` and `foldl`

We can use the Log monad to see how `foldr` and `foldl` iterate over the elements of a list. 

Here is the code for `foldr`:

```-- if the list is empty, the result is the initial value z; else 
-- apply f to the first element and the result of folding the rest 
foldr f z [] = z
foldr f z (x:xs) = f x (foldr f z xs)```

We refactor this in more monadic style: 

In [18]:
foldr' f z [] = z
foldr' f z (x:xs) = 
  do
    a <- foldr' f z xs
    f x a   

In [19]:
:t foldr'

foldr' :: forall (m :: * -> *) t1 t2. Monad m => (t1 -> t2 -> m t2) -> m t2 -> [t1] -> m t2

In [21]:
foldr' logAdd (logInit 10) [1..4]

Log "initial value = 10; add 4; add 3; add 2; add 1; " 20

Or if we want to track the calls to the accumulator function, we redefine the Log produced by `logAdd`: 

In [26]:
logPlus x y = Log (show x ++ "+" ++ show y ++ "; ") (x+y)

In [28]:
foldr' logPlus (logInit 10) [1..4]

Log "initial value = 10; 4+10; 3+14; 2+17; 1+19; " 20

As we might expect, the types don't work with `foldr` iteself: 

In [29]:
foldr logPlus (logInit 10) [1..4]

: 

We can handle `foldl` similarly. 

Here is the code for `foldl`:

```-- if the list is empty, the result is the initial value; else
-- we recurse immediately, making the new initial value the result 
-- of combining the old initial value with the first element. 
foldlf z [] = z
foldl f z (x:xs) = foldl f (f z x) xs```

The translation into monadic style is: 

In [31]:
foldl' f z [] = z
foldl' f z (x:xs) = 
  do
    z' <- z   
    let a = f z' x
    foldl' f a xs

In [32]:
foldr' logPlus (logInit 10) [1..4]
foldl' logPlus (logInit 10) [1..4]

Log "initial value = 10; 4+10; 3+14; 2+17; 1+19; " 20

Log "initial value = 10; 10+1; 11+2; 13+3; 16+4; " 20

In [ ]:
:t foldl'

In [ ]:
:t foldr'

So we can see that foldl and foldt work through the list in opposite directions. 

## Exercises